In [77]:
import pandas as pd
train_data = pd.read_csv('train_vidya3.csv')
test_data = pd.read_csv('test_vidya3.csv')
import numpy as np
shuffled_indices = np.random.permutation(train_data.shape[0])
train_data = train_data.iloc[shuffled_indices]

In [78]:
sum(pd.isna(test_data["Customer_Existing_Primary_Bank_Code"])==True)

4037

In [79]:
mode1 = train_data["City_Category"].mode()
train_data["City_Category"].fillna(mode1[0],inplace = True)

In [80]:
mode2 = train_data["Employer_Category1"].mode()
train_data["Employer_Category1"].fillna(mode2[0],inplace = True)

In [81]:
mode3 = train_data["Primary_Bank_Type"].mode()
train_data["Primary_Bank_Type"].fillna(mode3[0],inplace = True)

In [82]:
mode4 = train_data["DOB"].mode()
train_data["DOB"].where(pd.isnull(train_data["DOB"])==False,mode4[0],inplace=True)

In [83]:
train_data['DOB'] = train_data['DOB'].apply(lambda x: int(str(x)[-2:]))

In [84]:
train_data['Lead_Creation_Date'] = train_data['Lead_Creation_Date'].apply(lambda x: int(str(x)[3:5]))*100+train_data['Lead_Creation_Date'].apply(lambda x: int(str(x)[:2]))

In [85]:
mode5 = train_data["Employer_Category2"].mode()
train_data["Employer_Category2"].fillna(mode5[0],inplace = True)

In [86]:
train_data["Existing_EMI"].fillna(545436.500000,inplace = True)

In [87]:
mode6 = train_data["Customer_Existing_Primary_Bank_Code"].mode()
train_data["Customer_Existing_Primary_Bank_Code"].fillna(mode6[0],inplace = True)

In [88]:
train_data['Customer_Existing_Primary_Bank_Code'] = train_data['Customer_Existing_Primary_Bank_Code'].apply(lambda x: int(str(x)[-3:]))

In [89]:
test_data["City_Category"].fillna(mode1[0],inplace = True)

In [90]:
test_data["Employer_Category1"].fillna(mode2[0],inplace = True)

In [91]:
test_data["Primary_Bank_Type"].fillna(mode3[0],inplace = True)

In [92]:
test_data["DOB"].where(pd.isnull(test_data["DOB"])==False,mode4[0],inplace=True)

In [93]:
test_data['DOB'] = test_data['DOB'].apply(lambda x: int(str(x)[-2:]))

In [94]:
test_data['Lead_Creation_Date'] = test_data['Lead_Creation_Date'].apply(lambda x: int(str(x)[3:5]))*100+test_data['Lead_Creation_Date'].apply(lambda x: int(str(x)[:2]))

In [95]:
test_data["Employer_Category2"].fillna(mode5[0],inplace = True)

In [96]:
test_data["Existing_EMI"].fillna(545436.500000,inplace = True)

In [105]:
test_data["Customer_Existing_Primary_Bank_Code"].fillna(mode6[0],inplace = True)
test_data['Customer_Existing_Primary_Bank_Code'] = test_data['Customer_Existing_Primary_Bank_Code'].apply(lambda x: int(str(x)[-3:]))

In [220]:
y_train = train_data["Approved"].copy()
from sklearn.base import TransformerMixin,BaseEstimator
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self,attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values
num_attribs = ["DOB","Lead_Creation_Date","Employer_Category2","Monthly_Income","Existing_EMI","Var1","Customer_Existing_Primary_Bank_Code","Loan_Amount","Loan_Period"]
from sklearn.preprocessing import LabelBinarizer
class LabelBinarizer_new(TransformerMixin, BaseEstimator):
    def fit(self, X, y = 0):
        self.encoder = None
        return self
    def transform(self, X, y = 0):
        if(self.encoder is None):
            print("Initializing encoder")
            self.encoder = LabelBinarizer();
            result = self.encoder.fit_transform(X)
        else:
            result = self.encoder.transform(X)
        return result; 
from sklearn.pipeline import Pipeline
num_pipeline = Pipeline([("selector",DataFrameSelector(num_attribs))])
cat_pipeline1 = Pipeline([("selector",DataFrameSelector("Gender")),("label_binarizer",LabelBinarizer_new())])
cat_pipeline2 = Pipeline([("selector",DataFrameSelector("City_Category")),("label_binarizer",LabelBinarizer_new())])
cat_pipeline3 = Pipeline([("selector",DataFrameSelector("Employer_Category1")),("label_binarizer",LabelBinarizer_new())])
cat_pipeline4 = Pipeline([("selector",DataFrameSelector("Contacted")),("label_binarizer",LabelBinarizer_new())])
cat_pipeline5 = Pipeline([("selector",DataFrameSelector("Source_Category")),("label_binarizer",LabelBinarizer_new())])
cat_pipeline6 = Pipeline([("selector",DataFrameSelector("Primary_Bank_Type")),("label_binarizer",LabelBinarizer_new())])
from sklearn.pipeline import FeatureUnion
full_pipeline = FeatureUnion(transformer_list = [("num_pipeline",num_pipeline),("cat_pipeline1",cat_pipeline1),("cat_pipeline2",cat_pipeline2),("cat_pipeline3",cat_pipeline3),("cat_pipeline4",cat_pipeline4),("cat_pipeline5",cat_pipeline5),("cat_pipeline6",cat_pipeline6)])
X_train = full_pipeline.fit_transform(train_data)

Initializing encoder
Initializing encoder
Initializing encoder
Initializing encoder
Initializing encoder
Initializing encoder


In [99]:
from sklearn.ensemble import RandomForestClassifier

In [221]:
forest_clf = RandomForestClassifier(n_estimators=1000)

In [222]:
from sklearn.model_selection import cross_val_score

In [223]:
cross_val_score(forest_clf,X_train,y_train,cv=3,scoring="roc_auc")

array([0.80533472, 0.79810175, 0.80880555])

In [224]:
X_test = full_pipeline.transform(test_data)

In [225]:
forest_clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [288]:
#y_pred = forest_clf.predict(X_test)
ids = test_data['ID'].values
y_pred = pd.DataFrame(np.c_[ids,y_pred],columns=["ID","Approved"])
y_pred.to_csv('submission_vidya3.csv',index=False)

In [111]:
from sklearn.linear_model import LogisticRegression

In [235]:
log_reg = LogisticRegression()

In [236]:
log_reg.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [250]:
y_proba = log_reg.predict_proba(X_test)
y_proba

array([[0.91280231, 0.08719769],
       [0.99326452, 0.00673548],
       [0.9750027 , 0.0249973 ],
       ...,
       [0.98623029, 0.01376971],
       [0.98616179, 0.01383821],
       [0.98167464, 0.01832536]])

In [275]:
y_pred = (y_proba[:,1]>=0.015).astype(int)

In [276]:
sum(y_pred)

19393

In [227]:
y_proba = forest_clf.predict_proba(X_test)

In [213]:
mean1 = train_data["Loan_Amount"].mean()
#mean1
train_data["Loan_Amount"].fillna(mean1,inplace = True)

In [217]:
mode7 = train_data["Loan_Period"].mode()
#mean1
train_data["Loan_Period"].fillna(mode7[0],inplace = True)

In [219]:
test_data["Loan_Amount"].fillna(mean1,inplace = True)
test_data["Loan_Period"].fillna(mode7[0],inplace = True)

In [284]:
from sklearn.svm import SVC

In [289]:
svc_clf = SVC(probability = True)

In [ ]:
svc_clf.fit(X_train,y_train)

In [ ]:
y_pred = svc_clf.predict(X_test)